In [6]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
from scipy.stats.stats import spearmanr
from scipy.optimize import minimize
from BTC_Alpha_func import *
from tqdm import tqdm
import os
%matplotlib inline

In [48]:
class signal_search(object):
    
    def __init__(self, price, volume, freq = '10S', target = 30):
        self.init_price = price
        self.init_volume = volume
        self.diff_price = self.differential_diff(price)
        self.diff_volume = self.differential_diff(volume)
        self.adjust_freq(freq, target)
        self.circle = list(map(lambda x:int(x), list(np.hstack([np.linspace(0,119,120), 
                            np.linspace(120,180,11), np.linspace(180,720,19)[1:]]))))
    
    def differential_diff(self, init_price, d = 0.15, method = 'terms', terms_num = 10, thresh = 0.01):
        if method == 'thresh':
            k = 0
            coef = 1
            res = 0
            while coef < thresh:
                res += coef * init_price.shift(k).fillna(0)
                k += 1
                coef = -coef * (d-k+1)/k
        if method == 'terms':
            k = 0
            coef = 1
            res = 0
            while k < terms_num:
                res += coef * init_price.shift(k).fillna(0)
                k += 1
                coef = -coef * (d-k+1)/k
        return res
    
    def adjust_freq(self, freq = '10S', target = 30):
        
        
        idx = pd.IndexSlice
        
        #df.loc[idx[:, :, 'C1', :],:]
        
        self.high = self.init_price.resample(freq, label='right', closed='right').max()
        self.low = self.init_price.resample(freq, label='right', closed='right').min()
        self.close = self.init_price.resample(freq, label='right', closed='right').last()
        self.open = self.init_price.resample(freq, label='right', closed='right').first()
        self.volume = self.init_volume.resample(freq, label='right', closed='right').sum()
        self.vwap = pd.concat([self.init_price, self.init_volume], axis = 1).resample('60S', 
                    label='right', closed='right').apply(lambda x:np.average(x.iloc[:,0].values, 
                    weights = x.iloc[:,0].values) if np.sum(x.iloc[:,0].values) != 0 else np.average(x.iloc[:,0].values))#.iloc[:,0]
        self.amount = self.vwap * self.volume
        
        self.diff_high = self.diff_price.resample(freq, label='right', closed='right').max()
        self.diff_low = self.diff_price.resample(freq, label='right', closed='right').min()
        self.diff_close = self.diff_price.resample(freq, label='right', closed='right').last()
        self.diff_open = self.diff_price.resample(freq, label='right', closed='right').first()
        self.diff_volume = self.diff_volume.resample(freq, label='right', closed='right').sum()
        self.diff_vwap = pd.concat([self.diff_price, self.diff_volume], axis = 1).resample('60S', 
                    label='right', closed='right').apply(lambda x:np.average(x.iloc[:,0].values, 
                    weights = x.iloc[:,0].values) if np.sum(x.iloc[:,0].values) != 0 else np.average(x.iloc[:,0].values))#.iloc[:,0]
        self.diff_amount = self.diff_vwap * self.diff_volume
        
        self.TARGET = (self.close.shift(-target) / self.close - 1).fillna(0)
            
    def greedy_optimize_signal(self, alpha_func, initial_values):
        from copy import deepcopy
        best_pearsonr = 0
        for param_num in range(len(initial_values)):
            for i in self.circle:
                try:
                    temp_params = deepcopy(initial_values)
                    temp_params[param_num] = i
                    temp_signal = alpha_func(temp_params)
                    temp_pearsonr = -abs(pearsonr(temp_signal, self.TARGET)[0])
                    if temp_pearsonr < best_pearsonr:
                        initial_values[param_num] = i
                        best_pearsonr = temp_pearsonr
                except:
                    continue
        return initial_values, -abs(pearsonr(alpha_func(initial_values), self.TARGET)[0])
                        
    def test_signal(self, initial_alpha_func, initial_values, method = 'Greedy'):
        
        def alpha_func(initial_values):
            return initial_alpha_func(initial_values).replace(np.inf, np.nan).replace(-np.inf, np.nan).fillna(0)
        
        global HIGH, LOW, CLOSE, OPEN, VOLUME, VWAP, RET, AMOUNT, DTM, DBM, TR, HD, LD
        
        HIGH = self.high
        LOW = self.low
        CLOSE = self.close
        OPEN = self.open
        VOLUME = self.volume
        VWAP = self.vwap
        RET = (CLOSE / CLOSE.shift(1) - 1).fillna(0)
        MOUNT = self.amount
        DTM = (OPEN <= DELAY(OPEN, 1)) * MAX(HIGH - OPEN, OPEN - DELAY(OPEN, 1))
        DBM = (OPEN >= DELAY(OPEN, 1)) * MAX(OPEN - LOW,  OPEN - DELAY(OPEN, 1))
        TR = MAX(MAX(HIGH-LOW,ABS(HIGH-DELAY(CLOSE,1))),ABS(LOW-DELAY(CLOSE,1)) )
        HD = HIGH-DELAY(HIGH,1)
        LD = DELAY(LOW,1)-LOW

        if method == 'Greedy':
            best_normal_params, best_normal_pearsonr = self.greedy_optimize_signal(alpha_func, initial_values)
        elif method == 'ParticleSwarm':
            best_normal_params, best_normal_pearsonr = self.greedy_optimize_signal(alpha_func, initial_values)
        
        HIGH = self.diff_high
        LOW = self.diff_low
        CLOSE = self.diff_close
        OPEN = self.diff_open
        VOLUME = self.diff_volume
        VWAP = self.diff_vwap
        RET = (CLOSE / CLOSE.shift(1) - 1).fillna(0)
        AMOUNT = self.diff_amount
        DTM = (OPEN <= DELAY(OPEN, 1)) * MAX(HIGH - OPEN, OPEN - DELAY(OPEN, 1))
        DBM = (OPEN >= DELAY(OPEN, 1)) * MAX(OPEN - LOW, OPEN - DELAY(OPEN, 1))
        TR = MAX(MAX(HIGH-LOW,ABS(HIGH-DELAY(CLOSE,1))),ABS(LOW-DELAY(CLOSE,1)) )
        HD = HIGH-DELAY(HIGH,1)
        LD = DELAY(LOW,1)-LOW  
        
        Diff_or_Not = 1
        
        if method == 'Greedy':
            best_diff_params, best_diff_pearsonr = self.greedy_optimize_signal(alpha_func, initial_values)
        elif method == 'ParticleSwarm':
            best_diff_params, best_diff_pearsonr = self.greedy_optimize_signal(alpha_func, initial_values)
            
        if best_diff_pearsonr < best_normal_pearsonr:
            best_params = best_diff_params
            best_pearsonr = best_diff_pearsonr
        else:
            best_params = best_normal_params
            best_pearsonr = best_normal_pearsonr
            Diff_or_Not = 0
            
        if not Diff_or_Not:
            HIGH = self.high
            LOW = self.low
            CLOSE = self.close
            OPEN = self.open
            VOLUME = self.volume
            VWAP = self.vwap
            RET = (CLOSE / CLOSE.shift(1) - 1).fillna(0)
            MOUNT = self.amount
            DTM = (OPEN <= DELAY(OPEN, 1)) * MAX(HIGH - OPEN, OPEN - DELAY(OPEN, 1))
            DBM = (OPEN >= DELAY(OPEN, 1)) * MAX(OPEN - LOW, OPEN - DELAY(OPEN, 1))
            TR = MAX(MAX(HIGH-LOW,ABS(HIGH-DELAY(CLOSE,1))),ABS(LOW-DELAY(CLOSE,1)) )
            HD = HIGH-DELAY(HIGH,1)
            LD = DELAY(LOW,1)-LOW
            
        modify_type, modify_params, best_pearsonr = self.possible_modification(alpha_func, 
                                                                best_params, best_pearsonr)
        return (Diff_or_Not, modify_type, modify_params, best_params, best_pearsonr)
            
    def possible_modification(self, alpha_func, best_params, best_pearsonr):
        modify_type = ''
        modify_params = None
            
        #RANK
        signal = RANK(alpha_func(best_params))
        rank_pearsonr = -abs(pearsonr(signal, self.TARGET)[0])
        if rank_pearsonr < best_pearsonr:
            best_pearsonr = rank_pearsonr
            modify_type = 'RANK'
            modify_params = None
            
        #DIFF
        for d in np.linspace(0.05,1,20):
            signal = self.differential_diff(alpha_func(best_params), d = d, 
                                            method = 'terms', terms_num = 10, thresh = 0.01)
            diff_pearsonr = -abs(pearsonr(signal, self.TARGET)[0])
            if diff_pearsonr < best_pearsonr:
                best_pearsonr = diff_pearsonr
                modify_params = d
                modify_type = 'DIFF'
            else:
                break
            
        #EWMA
        if modify_type == 'RANK':
            best_signal = RANK(alpha_func(best_params))
        elif modify_type == 'DIFF':
            best_signal = self.differential_diff(alpha_func(best_params), d = modify_params, 
                                                 method = 'terms', terms_num = 10, thresh = 0.01)
        else:
            best_signal = alpha_func(best_params)
        
        ewma_or_not = 0
        best_half_life = 0
        for half_life in [1,3,6,12,18,30,60,90,180,360,720]:
            signal = best_signal.ewm(halflife = half_life).mean()
            ewma_pearsonr = -abs(pearsonr(signal, self.TARGET)[0])
            if ewma_pearsonr < best_pearsonr:
                best_pearsonr = ewma_pearsonr
                ewma_or_not = 1
                best_half_life = half_life
            else:
                break
        if ewma_or_not:
            modify_type += 'EWMA'
            modify_params = (modify_params, best_half_life)
                
        return modify_type, modify_params, best_pearsonr

In [49]:
trx = pd.read_pickle('trx_10s.pickle')
trx = trx.iloc(axis=0)[:int(2/3*len(trx))]

In [50]:
ss = signal_search(trx['CLOSE'], trx['VOLUME'])

In [51]:
def Alpha6(para_list):
#    assert (para_list[0] + para_list[1] == 1)
    return (RANK(SIGN(DELTA((((OPEN * para_list[0]) + (HIGH * (1-para_list[0])))), para_list[1])))* (-1))

In [52]:
ss.test_signal(Alpha6, np.array([1,2]))

(0, 'DIFF', 0.15, array([ 0, 76]), -0.06812079862455751)